# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.steps import AutoMLStep

from azureml.widgets import RunDetails

import joblib

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0




### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'heart-failure-ml-experiment'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

quick-starts-ws-269450
aml-quickstarts-269450
southcentralus
f5091c60-1c3c-430f-8d81-d802f6bf2414


## Configure Cluster

In [5]:
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "training"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Poll for a minimum number of nodes (min_nodes = 1). 
    # If no min node count is provided it uses the scale settings for the cluster.
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute cluster...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded............
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-11-03T15:26:12.300000+00:00', 'errors': None, 'creationTime': '2024-11-03T15:25:07.711924+00:00', 'modifiedTime': '2024-11-03T15:25:17.947349+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'Standard_DS3_v2'}


## Dataset

In [12]:
dataset_name = 'heart-failure-data'
dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [13]:
df = dataset.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

Field to predict - DEATH_EVENT which is a classification problem, we choose accuracy to be our primary metric here, as data is already registered we have the training dataset ready to use AutoML on it.

"primary_metric": "accuracy" = This specifies that the primary metric used to evaluate the performance of the models is "accuracy". Accuracy is the ratio of the number of correct predictions to the total number of predictions. It is commonly used for classification tasks where the model needs to predict discrete labels.

"experiment_timeout_minutes": 15 = This sets a time limit for the entire AutoML experiment, which is 15 minutes in this case. The experiment will run for a maximum of 15 minutes. If it doesn't finish within this time, it will stop regardless of whether it has tested all possible configurations or not. This is useful for ensuring experiments do not run indefinitely and helps in managing computational resources.

"max_concurrent_iterations": 5 = This specifies the maximum number of iterations (model training runs) that can be executed concurrently. Running multiple iterations in parallel can speed up the experiment by utilizing available computational resources efficiently. In this case, up to 5 iterations can run simultaneously.

In [14]:
target_column = "DEATH_EVENT"

# automl experiment settings here
automl_settings = {
    "primary_metric": "accuracy",
    "experiment_timeout_minutes": 18,
    "max_concurrent_iterations": 5,
    
}

# automl config here
automl_config = AutoMLConfig(
    compute_target= compute_target,
    training_data= dataset,
    task= "classification",
    label_column_name= target_column,
    path= "./automl-run",
    featurization= "auto",
    debug_log= "automl_errors.log",
    enable_early_stopping= True,
    **automl_settings
    )

In [15]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-ml-experiment,AutoML_43804fb4-dcd5-4883-8ab5-c76c3e5813ac,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [18]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

2024-11-03 15:34:43.909711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-03 15:34:45.450560: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-03 15:34:45.896715: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 15:34:50.150874: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-03 15:34:58.374461: W tensorflow/compiler/tf2

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-ml-experiment,AutoML_43804fb4-dcd5-4883-8ab5-c76c3e5813ac,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn more about cross validation: https://aka.ms/AutomatedMLCrossValidation
DETAILS:      
+------

{'runId': 'AutoML_43804fb4-dcd5-4883-8ab5-c76c3e5813ac',
 'target': 'training',
 'status': 'Completed',
 'startTimeUtc': '2024-11-03T15:33:51.668257Z',
 'endTimeUtc': '2024-11-03T15:51:16.322354Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'training',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-ml-experiment","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-269450","workspace_name":"quick-starts-ws-269450","region":"southcentralus","compute_target":"trainin

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [19]:
best_run, best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()["runDefinition"]["arguments"]

print(f"Best Model Run ID: {best_run.id}\n")
print(f"Best Model Metrics: {best_run_metrics}\n")
print(f"Best Model Accuracy: {best_run_metrics['accuracy']}\n")

Best Model Run ID: AutoML_43804fb4-dcd5-4883-8ab5-c76c3e5813ac_38

Best Model Metrics: {'average_precision_score_micro': 0.9276191394072031, 'precision_score_macro': 0.8928920414810507, 'recall_score_macro': 0.8557813320356227, 'log_loss': 0.3479908410470068, 'weighted_accuracy': 0.9051248288688786, 'average_precision_score_weighted': 0.9290562483279128, 'recall_score_weighted': 0.8863218390804597, 'AUC_macro': 0.9154500663714623, 'recall_score_micro': 0.8863218390804597, 'accuracy': 0.8863218390804597, 'average_precision_score_macro': 0.9058437542309316, 'precision_score_micro': 0.8863218390804597, 'f1_score_macro': 0.8630495955722551, 'f1_score_micro': 0.8863218390804597, 'norm_macro_recall': 0.7115626640712452, 'f1_score_weighted': 0.8831156839527352, 'precision_score_weighted': 0.8971886610309531, 'AUC_weighted': 0.9154500663714623, 'balanced_accuracy': 0.8557813320356227, 'AUC_micro': 0.9246084026952042, 'matthews_correlation': 0.7453466933174661, 'confusion_matrix': 'aml://artifa

In [20]:
print(f"More details on best Model {best_model}")
# save the best model
best_run = remote_run.get_best_child()
best_model_name = best_run.properties["model_name"]
best_run.download_files("./outputs")


More details on best Model Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook269450/code/Users/odl_user_269450')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('18', Pipeli...e='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))]))], flatten_transform=False, weights=[0.2222222222222222, 0.2222222222222222, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111]))])


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
best_run.get_file_names()
# registering the model
from azureml.core.resource_configuration import ResourceConfiguration
best_model_name = best_run.properties["model_name"]
model = best_run.register_model(
    model_name=best_model_name, 
    model_path="./outputs",
    resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1),
)

print(f"Model name: {model.name}")
print(f"Version: {model.version}")
print(f"RunID: {model.run_id}")


Model name: AutoML43804fb4d38
Version: 1
RunID: AutoML_43804fb4-dcd5-4883-8ab5-c76c3e5813ac_38


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=3, memory_gb=15)

inference_env = Environment.get(ws, "AzureML-AutoML")


inference_config = InferenceConfig(entry_script="./score.py", environment=inference_env)


service = Model.deploy(ws, "heart-beat-endpoint", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)
service.scoring_uri

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-11-03 15:59:18+00:00 Creating Container Registry if not exists..
2024-11-03 16:09:18+00:00 Registering the environment.
2024-11-03 16:09:20+00:00 Use the existing image..
2024-11-03 16:09:25+00:00 Submitting deployment to compute..
2024-11-03 16:09:31+00:00 Checking the status of deployment heart-beat-endpoint..
2024-11-03 16:17:44+00:00 Checking the status of inference endpoint heart-beat-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


'http://6109292e-50b5-4b0c-b85d-ceaf7da2628c.southcentralus.azurecontainer.io/score'

In [25]:
service.update(enable_app_insights=True)
service.get_logs()

'2024-11-03T16:17:31,675777700+00:00 - rsyslog/run \n2024-11-03T16:17:31,685193400+00:00 - gunicorn/run \n2024-11-03T16:17:31,688854100+00:00 | gunicorn/run | \n2024-11-03T16:17:31,692850600+00:00 | gunicorn/run | ###############################################\n2024-11-03T16:17:31,695937100+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-11-03T16:17:31,704274300+00:00 | gunicorn/run | ###############################################\n2024-11-03T16:17:31,713776600+00:00 - nginx/run \n2024-11-03T16:17:31,716430000+00:00 | gunicorn/run | \n2024-11-03T16:17:31,724938600+00:00 | gunicorn/run | \n2024-11-03T16:17:31,732716800+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240818.v1\n2024-11-03T16:17:31,736931800+00:00 | gunicorn/run | \n2024-11-03T16:17:31,742695300+00:00 | gunicorn/run | \n2024-11-03T16:17:31,748304100+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

TODO: In the cell below, print the logs of the web service and delete the service

In [27]:
import urllib.request
import json
import os

data =  {
  "data": [
    {
      "age": 25.0,
      "anaemia": 0,
      "creatinine_phosphokinase": 0,
      "diabetes": 1,
      "ejection_fraction": 0,
      "high_blood_pressure": 1,
      "platelets": 0.0,
      "serum_creatinine": 0.0,
      "serum_sodium": 0,
      "sex": 1,
      "smoking": 1,
      "time": 0
    }
  ]
}

body = str.encode(json.dumps(data))

url = 'http://6109292e-50b5-4b0c-b85d-ceaf7da2628c.southcentralus.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[1]'


In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
